## 버스만 있는 이미지

In [1]:
import pandas as pd

valid_total_df = pd.read_csv('valid_total_txt_label.csv')

condition1 = valid_total_df['car_cnt'].isnull()
condition2 = valid_total_df['truck_cnt'].isnull()

busonlyList = list(valid_total_df[(condition1) & (condition2)]['txt_in_valid_folder'].values)

In [8]:
import os

valid_filename = []
for i in busonlyList:
    basename = os.path.basename(i)
    valid_filename.append(basename)
valid_filename

['Suwon_CH05_20200720_1830_MON_9m_NH_highway_OW5_sunny_FHD_066.txt',
 'Suwon_CH05_20200720_1830_MON_9m_NH_highway_OW5_sunny_FHD_085.txt',
 'Suwon_CH06_20200722_1900_WED_9m_NH_highway_OW5_sunny_FHD_033.txt',
 'Suwon_CH10_20201213_0752_SUN_9m_NH_highway_OW5_snow_FHD_041.txt',
 'Suwon_CH10_20201213_0752_SUN_9m_NH_highway_OW5_snow_FHD_082.txt']

In [4]:
from glob import glob
import cv2
import numpy as np


def glob_files(path, file_type="*"):
    search_string = os.path.join(path, file_type)
    files = glob(search_string)

    # print('searching ', path)
    paths = []
    for f in files:
      if os.path.isdir(f):
        sub_paths = glob_files(f + '/')
        paths += sub_paths
      else:
        paths.append(f)

    # We sort the images in alphabetical order to match them
    #  to the annotation files
    paths.sort()

    return paths


# IMAGE_SIZE = 600

def load_images(path):
  files = glob_files(path, "*.png")

  # print(files)
  X_data = []
  for file in files:
    image = cv2.imread(file)
    # print(image.shape)
    # x = cv2.resize(image, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

    X_data.append(image)
    
  return np.array(X_data)

In [7]:
basePath_image = 'highway/valid/image/'

X_test = load_images(basePath_image)
print(X_test.shape)

(5, 1920, 1080, 3)


In [9]:
WIDTH = 1080
HEIGHT = 1920

def load_labels(path):
  files = glob_files(path, "*.txt")

  Y_data = []
  for file in files:
    with open(file) as f:
      lines = f.readlines()

      boxes = []
      for line in lines:
        tokens = line.split()

        class_id = int(tokens[0])
        xc = float(tokens[1]) * WIDTH
        yc = float(tokens[2]) * HEIGHT
        width = float(tokens[3]) * WIDTH
        height = float(tokens[4]) * HEIGHT

        boxes.append(np.array([class_id, xc, yc, width, height]))
        # print(class_id, xc, yc, width, height)

      Y_data.append(np.array(boxes))
      # print(lines)
  return Y_data  ## 이미지마다 탐지 대상 개수가 달라서(행 개수가 달라서) 정해진 shape의 넘파이 어레이로 변환할 수 없다! 변환하려면 패딩 등이 필요.


## car, truck, bus 가 모두 포함된 이미지

In [20]:
condition3 = valid_total_df['car_cnt'].notnull()
condition4 = valid_total_df['truck_cnt'].notnull()
condition5 = valid_total_df['bus_cnt'].notnull()

valid_notnull_df = valid_total_df[(condition3) & (condition4) & (condition5)]
valid_notnull_df[valid_notnull_df['car_cnt'] <= 10]

,txt_in_valid_folder,car_cnt,truck_cnt,bus_cnt
2,highway/valid/label/Suwon_CH01_20200720_1830_M...,5.0,9.0,1.0
3,highway/valid/label/Suwon_CH01_20200720_1830_M...,8.0,7.0,1.0
5,highway/valid/label/Suwon_CH01_20200720_1830_M...,9.0,4.0,1.0
6,highway/valid/label/Suwon_CH01_20200720_1830_M...,9.0,4.0,1.0
7,highway/valid/label/Suwon_CH01_20200720_1830_M...,7.0,4.0,3.0
...,...,...,...,...
9290,highway/valid/label/Suwon_CH10_20201213_0752_S...,2.0,1.0,1.0
9311,highway/valid/label/Suwon_CH10_20201213_0752_S...,2.0,2.0,1.0
9312,highway/valid/label/Suwon_CH10_20201213_0752_S...,1.0,1.0,1.0
9315,highway/valid/label/Suwon_CH10_20201213_0752_S...,1.0,1.0,1.0
